In [ ]:
import math
import numpy
import ipywidgets
from matplotlib import pyplot
%matplotlib inline

In [ ]:
import sys
modules_dir = '../src/python'
if modules_dir not in sys.path:
    sys.path.insert(0, modules_dir)

from regularize import regularize
import flapping

In [ ]:
# Set the type and size of the font to use in Matplotlib figures.
pyplot.rc('font', family='serif', size=16)

In [ ]:
def ellipse(a, b, center=(0.0, 0.0), num=50):
    """
    Returns the coordinates of an ellipse.
    
    Parameters
    ----------
    a: float
        The semi-major axis of the ellipse.
    b: float
        The semi-minor axis of the ellipse.
    center: 2-tuple of floats, optional
        The position of the center of the ellipse;
        default: (0.0, 0.0)
    num: integer, optional
        The number of points on the upper side of the ellipse.
        The number includes the leading and trailing edges.
        Thus, the total number of points will be 2 * (num - 1);
        default: 50.
    
    Returns
    -------
    x: numpy.ndarray of floats
        The x-coordinates of the ellipse.
    y: numpy.ndarray of floats
        The y-coordinates of the ellipse.
    """
    xc, yc = center
    x_upper = numpy.linspace(xc + a, xc - a, num=num)
    y_upper = b / a * numpy.sqrt(a**2 - x_upper**2)
    x_lower = numpy.linspace(xc - a, xc + a, num=num)[1:-1]
    y_lower = -b / a * numpy.sqrt(a**2 - x_lower**2)
    x = numpy.concatenate((x_upper, x_lower))
    y = numpy.concatenate((y_upper, y_lower))
    return x, y

In [ ]:
def kinematics(x0, y0, center, t, flapping):
    alpha = flapping.orientation_angle(t)
    x, y = rotate(x0, y0, center=center, angle=alpha)
    xd, yd = flapping.displacement(t)
    x += xd
    y += yd
    return x, y

In [ ]:
# Create an ellipse.
c = 1.0  # chord of the ellipse (major axis)
r = 0.1  # ratio between minor and major axis
a, b = c / 2.0, r * c / 2.0
x0, y0 = ellipse(a, b, center=(0.0, 0.0), num=100)
ds = 0.025  # target distance between two consecutive points.
x0, y0 = regularize(x0, y0, ds=ds)

# Plot the ellipse.
fig, ax = pyplot.subplots(figsize=(6.0, 6.0))
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.grid()
ax.plot(x0, y0, color='C0', linestyle='-', linewidth=2)
ax.axis('scaled', adjustable='box')
ax.set_xlim(-0.75, 0.75)
ax.set_ylim(-0.25, 0.25);

In [ ]:
kinematics = flapping.Flapping()

In [ ]:
def plot_position(t, x0, y0, kinematics):
    t /= kinematics.f
    x, y = kinematics.position(t, x0, y0)
    fig, ax = pyplot.subplots(figsize=(8.0, 8.0))
    ax.set_title('t / T = {}'.format(t * kinematics.f))
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.grid()
    ax.plot(x, y)
    ax.axis('scaled', adjustable='box')
    ax.set_xlim(-2.0, 2.0)
    ax.set_ylim(-1.0, 1.0)
    fig.tight_layout()

In [ ]:
# Create and display an interative plot of the kinematics.
time_slider = ipywidgets.FloatSlider(value=0.0, min=0.0, max=4.0,
                                     step=0.05, description='t / T')
w = ipywidgets.interactive(plot_position, t=time_slider,
                           x0=ipywidgets.fixed(x0),
                           y0=ipywidgets.fixed(y0),
                           kinematics=ipywidgets.fixed(kinematics))
display(w)

In [ ]:
t_nodim = numpy.linspace(0.0, 0.5, num=100)
t = t_nodim / kinematics.f
alpha = kinematics.orientation_angle(t)
cd, cl = kinematics.quasi_steady_coefficients(t)
alpha_deg = numpy.degrees(alpha)

fig, ax = pyplot.subplots(figsize=(6.0, 6.0))
ax.set_xlabel(r'$\alpha$ (deg)')
ax.set_ylabel('Quasi-steady force coefficients')
ax.grid()
ax.plot(alpha_deg, cd, label='$C_D$')
ax.plot(alpha_deg, cl, label='$C_L$')
ax.legend()
ax.set_ylim(-0.5, 2.5);

In [ ]:
D, L = [], []
for ti in t:
    alpha = kinematics.orientation_angle(ti)
    x, y = flapping.rotate(x0, y0, center=(0.0, 0.0), angle=alpha)
    Di, Li = kinematics.quasi_steady_forces(ti, x, numpy.zeros_like(x), 0.0, 0.0, rho=1.0)
    D.append(numpy.max(Di))
    L.append(numpy.max(Li))
D, L = numpy.array(D), numpy.array(L)

fig, ax = pyplot.subplots(figsize=(6.0, 6.0))
ax.set_xlabel('$t / T$')
ax.set_ylabel('Quasi-steady forces')
ax.grid()
ax.plot(t_nodim, D, label='Drag')
ax.plot(t_nodim, L, label='Lift')
ax.set_xlim(t_nodim[0], t_nodim[-1])
ax.legend();